# Configuration

In [1]:
from ast import literal_eval
import joblib

import dotenv
dotenv.load_dotenv()

import itertools
from time import sleep

import pandas as pd

from jinja2 import Template
from typing import Dict, Any
from gemini_llm import GeminiLLM
from chroma_client import ChromaDBManager

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from langchain_community.embeddings import SentenceTransformerEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_core.documents import Document
from ragas.testset.persona import Persona

from ragas.testset import TestsetGenerator
from ragas import evaluate, RunConfig
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, LLMContextPrecisionWithoutReference, AnswerRelevancy, AnswerCorrectness, AnswerSimilarity, Faithfulness
from datasets import Dataset  

In [2]:
PROMPT_TEMPLATE = """
{# Main sections #}
# Scene summaries
{{ story }}

# Relevant text excerpts
{{ search_results }}

{# Instructions for using sections #}
When answering questions:
1. Use "Search results" for:
  - Direct character quotes
  - Specific event/situation details
  - Exact dialogue details 
  - Text fact verification

2. Use "Scene summaries" only for:
  - Understanding overall context
  - Clarifying event timeline
  - Connecting different text parts

{# User question #}
Question: {{ user_question }}
"""

In [3]:
class RagTester:
    def __init__(self):
        self.chroma_manager = ChromaDBManager()
        self.llm = GeminiLLM()
        self.template = Template(PROMPT_TEMPLATE)
        self.chat_history = []

    def create_prompt(self, collection_name: str, query: str) -> str:
        """
        Создает промпт на основе истории, результатов поиска и вопроса пользователя
        """
        story = self.chroma_manager.get_complete_story(collection_name)
        search_results = self.chroma_manager.search(collection_name, query)

        prompt = self.template.render(
            story=story,
            search_results=search_results,
            user_question=query
        )

        return search_results, prompt

    def get_response(self, collection_name: str, query: str) -> Dict[str, Any]:
        """
        Получает ответ от модели с учетом контекста коллекции
        """
        search_results, full_prompt = self.create_prompt(
            collection_name=collection_name,
            query=query
        )
        
        response = self.llm.generate_response(
            full_prompt,
            chat_history=self.chat_history
        )
        
        # Обновляем историю чата
        self.chat_history.extend([
            {"role": "user", "parts": [{"text": query}]},
            {"role": "assistant", "parts": [{"text": response["text"]}]}
        ])
        
        return search_results, response

In [4]:
tester = RagTester()

<All keys matched successfully>


In [5]:
collections = tester.chroma_manager.get_base_collection_names()
print("Available collections:", collections)

INFO:chroma_client:Найдено 3 базовых коллекций: ['Sherlock_Study_in_Scarlet', 'The-Shadow-of-the-Light-Fairy', 'The_Power_Of_Prophecy']


Available collections: ['Sherlock_Study_in_Scarlet', 'The-Shadow-of-the-Light-Fairy', 'The_Power_Of_Prophecy']


In [6]:
collection_name = "The-Shadow-of-the-Light-Fairy"

In [106]:
query = "How did Hadia and Taimur first meet?"  # Укажите ваш вопрос

search_context, response = tester.get_response(collection_name, query)
print("\nQuestion:", query)
print("\nContext:", search_context)
print("\nResponse:", response['text'])
print("\nTokens used:", response.get("tokens", "N/A"))

INFO:chroma_client:Собрано 57 сцен в единый текст
INFO:chroma_client:Поиск по запросу: How did Hadia and Taimur first meet?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Question: How did Hadia and Taimur first meet?

Context: [{'scene': {'document': 'search_document: Taimur and Hadia meet as interns at Imperial University and create a WhatsApp group.', 'metadata': {'chunk_index': 0, 'key_details': 'They meet accidentally, They create a WhatsApp group for interns', 'location': 'Imperial University', 'scene_index': 1, 'who': 'Taimur, Hadia'}}, 'chunks': [{'document': 'as a lover or not. It was the second time he unblocked her.\r\n\r\nNow, it seemed that Hadia was also absorbed in him, but it was\r\nnot sured. Next week Hadia gave good news to Taimur, “I will\r\ncome to university by tomorrow”.\r\n\r\nIt was a piece of breaking news for Taimur. He was very happy\r\nafter seeing that text.\r\n\r\nAlthough, they already met, but now there was another thing\r\nwhich was a heart-full connection from Taimur. But there was', 'metadata': {'global_chunk_index': 28, 'parent_chunk_index': 0, 'subchunk_index': 28}}, {'document': 'due to Taimur’s shyness, hesitatio

# Ragas

In [116]:
def parsing_context(context):
    
    retrieved_context = []
    
    for item_scene in context:
        if item_scene.get('chunks'):
            if item_scene['scene']['document'] != 'Global Search Results':
                retrieved_context.append(item_scene['scene']['document'])
            for chunk in item_scene['chunks']:
                retrieved_context.append(chunk['document'])
    
    return retrieved_context

## Generate

In [8]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
# generator_llm = LangchainLLMWrapper(llm)
# generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)
model_name = "nomic-ai/nomic-embed-text-v1.5"
embedding_model = SentenceTransformerEmbeddings(model_name=model_name, model_kwargs={"trust_remote_code":True, 'device': 'cuda'})
generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

/tmp/ipykernel_4266/3403442604.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name=model_name, model_kwargs={"trust_remote_code":True, 'device': 'cuda'})
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: nomic-ai/nomic-embed-text-v1.5


In [40]:
docs = []
count_chunks = tester.chroma_manager.client.get_collection(collection_name + '_chunks').count()

for doc in tester.chroma_manager.client.get_collection(collection_name + '_chunks').peek(count_chunks)['documents']:
    docs.append(Document(metadata={"file_name": collection_name},
                         page_content=doc))

In [48]:
personas = list(
    map(lambda x: Persona(name=x, role_description=''),
        set(
            list(
                itertools.chain.from_iterable(map(lambda x: x.split(', '),
                                                  [scene['who'] for scene in tester.chroma_manager.client.get_collection(collection_name + '_scenes').peek(100)['metadatas']]
                                                  )
                                              )
                )
            )
        )
)
personas

[Persona(name='Aliza', role_description=''),
 Persona(name='Gull', role_description=''),
 Persona(name='Hanzla', role_description=''),
 Persona(name='Faheem', role_description=''),
 Persona(name='interviewer', role_description=''),
 Persona(name='Mr. Zia', role_description=''),
 Persona(name='Aish', role_description=''),
 Persona(name='Ahsan', role_description=''),
 Persona(name='Samia', role_description=''),
 Persona(name='Sameer', role_description=''),
 Persona(name='Taimoor', role_description=''),
 Persona(name='Zaini', role_description=''),
 Persona(name='Saim', role_description=''),
 Persona(name='Hadia', role_description=''),
 Persona(name='Taimur', role_description='')]

In [57]:
testsetgenerator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

In [50]:
# testsetgenerator.knowledge_graph = joblib.load('testsetgenerator_kg.pkl')
# testsetgenerator.persona_list = joblib.load('testsetgenerator_pl.pkl')

In [58]:
testsetgenerator

TestsetGenerator(llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), embedding_model=LangchainEmbeddingsWrapper(embeddings=HuggingFaceEmbeddings(...)), knowledge_graph=KnowledgeGraph(nodes: 0, relationships: 0), persona_list=[Persona(name='Aliza', role_description=''), Persona(name='Gull', role_description=''), Persona(name='Hanzla', role_description=''), Persona(name='Faheem', role_description=''), Persona(name='interviewer', role_description=''), Persona(name='Mr. Zia', role_description=''), Persona(name='Aish', role_description=''), Persona(name='Ahsan', role_description=''), Persona(name='Samia', role_description=''), Persona(name='Sameer', role_description=''), Persona(name='Taimoor', role_description=''), Persona(name='Zaini', role_description=''), Persona(name='Saim', role_description=''), Persona(name='Hadia', role_description=''), Persona(name='Taimur', role_description='')])

In [59]:
docs

[Document(metadata={'file_name': 'The-Shadow-of-the-Light-Fairy'}, page_content='THE SHADOW                                                                               A FICTIONAL STORY FOR YOUNGSTERS…!\r\n\r\n            OF\r\n\r\n\r\n\r\n\r\n                                                                    THE SHADOW OF THE LIGHT FAIRY\r\n      THE LIGHT FAIRY                                                                                  THE SHADOW\r\n                                                                                                          OF'),
 Document(metadata={'file_name': 'The-Shadow-of-the-Light-Fairy'}, page_content='Sometimes it seems that we have lost\r\n                       Everything but we don’t. That is only the                                    THE LIGHT FAIRY\r\n                         Realization. Our soul demands a lot\r\n                        From us and that feeling is one of them\r\n                        That is necessary for spiritu

In [60]:
len(docs)

269

In [72]:
test_size = 150
testset = testsetgenerator.generate_with_langchain_docs(docs, testset_size=test_size)

Applying SummaryExtractor:   0%|          | 0/218 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Applying CustomNodeFilter:   0%|          | 0/269 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/756 [00:00<?, ?it/s]

...

INFO:ragas.testset.synthesizers.multi_hop.abstract:found 0 clusters
INFO:ragas.testset.synthesizers.multi_hop.specific:found 464 clusters


Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ragas.testset.synthesizers.multi_hop.specific:found 464 clusters
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Requ

Generating Samples:   0%|          | 0/150 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [73]:
testset_pd = testset.to_pandas()
testset_pd.drop(columns=['reference_contexts'], inplace=True)

In [74]:
testset_pd

,user_input,reference,synthesizer_name
0,What is the publication date of the First Edit...,The First Edition is scheduled for May 2024.,single_hop_specifc_query_synthesizer
1,What is the primary intent of the novella for ...,The primary intent of the novella is to impart...,single_hop_specifc_query_synthesizer
2,Where Taimoor Ahmad study in the story?,"Taimoor Ahmad studies at Imperial University, ...",single_hop_specifc_query_synthesizer
3,Who is Zaini in relation to Taimoor?,Zaini is a very intelligent girl who used to p...,single_hop_specifc_query_synthesizer
4,Who is Gull's best friend?,"Gull's best friend is Samia, who is also a cla...",single_hop_specifc_query_synthesizer
...,...,...,...
145,What were Taimoor's reasons for wanting to mee...,Taimoor wanted to meet Hadia before the univer...,multi_hop_specific_query_synthesizer
146,What impact did Hadia's experiences at the uni...,"Hadia's experiences at the university, where s...",multi_hop_specific_query_synthesizer
147,What significant event related to Taimoor's fe...,When Hadia announced she would come to univers...,multi_hop_specific_query_synthesizer
148,What role did Aliza play in Taimoor's situatio...,Aliza played a supportive role in Taimoor's si...,multi_hop_specific_query_synthesizer


In [75]:
testset_pd.to_csv('synthetic_data_ragas_4o-mini.csv', index=False)

In [76]:
# joblib.dump(testsetgenerator.knowledge_graph, 'testsetgenerator_kg_llama_4o-mini.pkl')

['testsetgenerator_kg_llama_4o-mini.pkl']

In [77]:
# joblib.dump(testsetgenerator.persona_list, 'testsetgenerator_pl_llama_4o-mini.pkl') 

['testsetgenerator_pl_llama_4o-mini.pkl']

## Eval

In [17]:
def get_answer(collection_name, user_input):
    retrieved_contexts, response = tester.get_response(collection_name, user_input)
    sleep(5)
    return retrieved_contexts, response['text']

In [18]:
testset_pd = pd.read_csv('synthetic_data_ragas_4o-mini.csv')

In [19]:
testset_pd

,user_input,reference,synthesizer_name
0,What is the publication date of the First Edit...,The First Edition is scheduled for May 2024.,single_hop_specifc_query_synthesizer
1,What is the primary intent of the novella for ...,The primary intent of the novella is to impart...,single_hop_specifc_query_synthesizer
2,Where Taimoor Ahmad study in the story?,"Taimoor Ahmad studies at Imperial University, ...",single_hop_specifc_query_synthesizer
3,Who is Zaini in relation to Taimoor?,Zaini is a very intelligent girl who used to p...,single_hop_specifc_query_synthesizer
4,Who is Gull's best friend?,"Gull's best friend is Samia, who is also a cla...",single_hop_specifc_query_synthesizer
...,...,...,...
145,What were Taimoor's reasons for wanting to mee...,Taimoor wanted to meet Hadia before the univer...,multi_hop_specific_query_synthesizer
146,What impact did Hadia's experiences at the uni...,"Hadia's experiences at the university, where s...",multi_hop_specific_query_synthesizer
147,What significant event related to Taimoor's fe...,When Hadia announced she would come to univers...,multi_hop_specific_query_synthesizer
148,What role did Aliza play in Taimoor's situatio...,Aliza played a supportive role in Taimoor's si...,multi_hop_specific_query_synthesizer


In [20]:
# список индексов вопросов, которые адекватны книге
index_good_question = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
 30, 31, 33, 34, 35, 36, 37, 38, 39, 40,
 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
 51, 52, 53, 54, 55, 57, 58, 59, 60, 61,
 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
 72, 73, 74, 75, 76, 77, 78, 79, 80, 82,
 83, 84, 85, 86, 87, 88, 89, 90, 91, 92,
 93, 94, 95, 96, 97, 98, 100, 101, 102, 103,
 104, 105, 106, 107, 108, 109, 111, 112, 113, 114,
 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
 145, 146, 147, 148, 149]

In [21]:
testset_pd = testset_pd.iloc[index_good_question].reset_index(drop=True).copy()

In [24]:
evalset_pd = pd.DataFrame({'user_input':testset_pd['user_input'],
                           'retrieved_contexts': [''] * len(testset_pd),
                           'response': [''] * len(testset_pd),
                           'reference': testset_pd['reference']})

In [25]:
evalset_pd[['retrieved_contexts', 'response']] = evalset_pd.apply(lambda row: get_answer(collection_name, row['user_input']), axis=1, result_type ='expand')

INFO:chroma_client:Собрано 57 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What is the publication date of the First Edition?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 57 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What is the primary intent of the novella for readers?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 57 сцен в единый текст
INFO:chroma_client:Поиск по запросу: Where Taimoor Ahmad study in the story?


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 57 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What role did Aliza play in Taimoor's situation with Hadia, and how is Ali related to her in terms of education and interests?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 57 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What did Ahsan say to Taimoor about Hadia during their park visit?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
evalset_pd.to_csv('synthetic_full_data_ragas_4o-mini_gemini.csv', index=False)

In [63]:
evalset_pd = pd.read_csv('synthetic_full_data_ragas_4o-mini_gemini.csv', converters={'retrieved_contexts': literal_eval})

In [64]:
evalset_pd

,user_input,retrieved_contexts,response,reference
0,What is the publication date of the First Edit...,[{'scene': {'document': 'search_document: Hadi...,Search results: First Edition: May 2024\n,The First Edition is scheduled for May 2024.
1,What is the primary intent of the novella for ...,[{'scene': {'document': 'search_document: Same...,Search results: The primary intent of the nove...,The primary intent of the novella is to impart...
2,Where Taimoor Ahmad study in the story?,[{'scene': {'document': 'search_document: Taim...,Search results: Taimoor Ahmad studies at Imper...,"Taimoor Ahmad studies at Imperial University, ..."
3,Who is Zaini in relation to Taimoor?,[{'scene': {'document': 'search_document: Taim...,"Based on the **scene summaries**, Taimoor meet...",Zaini is a very intelligent girl who used to p...
4,Who is Gull's best friend?,[{'scene': {'document': 'search_document: Taim...,Based on the **scene summaries** and **search ...,"Gull's best friend is Samia, who is also a cla..."
...,...,...,...,...
140,What were Taimoor's reasons for wanting to mee...,[{'scene': {'document': 'search_document: Taim...,"Okay, let's analyze Taimoor's reasons for want...",Taimoor wanted to meet Hadia before the univer...
141,What impact did Hadia's experiences at the uni...,[{'scene': {'document': 'search_document: Taim...,"Okay, let's analyze the impact of Hadia's expe...","Hadia's experiences at the university, where s..."
142,What significant event related to Taimoor's fe...,[{'scene': {'document': 'search_document: Taim...,Based on the **search results**:\n\nThe signif...,When Hadia announced she would come to univers...
143,What role did Aliza play in Taimoor's situatio...,[{'scene': {'document': 'search_document: Taim...,"Okay, let's break down Aliza's role in Taimoor...",Aliza played a supportive role in Taimoor's si...


In [65]:
evalset_pd['retrieved_contexts'] = evalset_pd.apply(lambda row: parsing_context(row['retrieved_contexts']), axis=1)

In [66]:
evalset_pd

,user_input,retrieved_contexts,response,reference
0,What is the publication date of the First Edit...,[Love you so…. Much Mr. T. (Taimoor’s nickname...,Search results: First Edition: May 2024\n,The First Edition is scheduled for May 2024.
1,What is the primary intent of the novella for ...,"[Taimoor, “Maybe but what difference does it m...",Search results: The primary intent of the nove...,The primary intent of the novella is to impart...
2,Where Taimoor Ahmad study in the story?,[man trying to convince a scholar who knows al...,Search results: Taimoor Ahmad studies at Imper...,"Taimoor Ahmad studies at Imperial University, ..."
3,Who is Zaini in relation to Taimoor?,[man trying to convince a scholar who knows al...,"Based on the **scene summaries**, Taimoor meet...",Zaini is a very intelligent girl who used to p...
4,Who is Gull's best friend?,[Gull encouraged and believed him that everyth...,Based on the **scene summaries** and **search ...,"Gull's best friend is Samia, who is also a cla..."
...,...,...,...,...
140,What were Taimoor's reasons for wanting to mee...,[June her friend Aliza was also in university ...,"Okay, let's analyze Taimoor's reasons for want...",Taimoor wanted to meet Hadia before the univer...
141,What impact did Hadia's experiences at the uni...,[special and attracted him to Hadia. Although ...,"Okay, let's analyze the impact of Hadia's expe...","Hadia's experiences at the university, where s..."
142,What significant event related to Taimoor's fe...,[June her friend Aliza was also in university ...,Based on the **search results**:\n\nThe signif...,When Hadia announced she would come to univers...
143,What role did Aliza play in Taimoor's situatio...,[and she (Aliza) was trying to misguide everyo...,"Okay, let's break down Aliza's role in Taimoor...",Aliza played a supportive role in Taimoor's si...


In [14]:
run_config = RunConfig(timeout=120, max_wait = 180, max_workers= 1)

In [15]:
score_result = evaluate(
  dataset=Dataset.from_pandas(evalset_pd),
  metrics=[
      LLMContextRecall(llm=generator_llm),
      LLMContextPrecisionWithReference(llm=generator_llm),
      LLMContextPrecisionWithoutReference(llm=generator_llm),
      AnswerCorrectness(llm=generator_llm, embeddings=generator_embeddings),
      AnswerRelevancy(llm=generator_llm, embeddings=generator_embeddings),
      AnswerSimilarity(embeddings=generator_embeddings),
      Faithfulness(llm=generator_llm)
  ]
)

Evaluating:   0%|          | 0/1015 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [16]:
score_result

{'context_recall': 0.7167, 'llm_context_precision_with_reference': 0.5525, 'llm_context_precision_without_reference': 0.7388, 'answer_correctness': 0.4453, 'answer_relevancy': 0.7584, 'semantic_similarity': 0.7353, 'faithfulness': 0.7294}

In [17]:
scoreset_pd = score_result.to_pandas()

In [53]:
scoreset_pd.to_csv('synthetic_score_data_ragas_4o-mini_gemini.csv', index=False)

In [67]:
scoreset_pd = pd.read_csv('synthetic_score_data_ragas_4o-mini_gemini.csv', converters={'retrieved_contexts': literal_eval})

In [68]:
scoreset_pd

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What is the publication date of the First Edit...,[Love you so…. Much Mr. T. (Taimoor’s nickname...,Search results: First Edition: May 2024\n,The First Edition is scheduled for May 2024.,1.000000,0.111111,0.111111,0.157921,0.733777,0.631684,NaN
1,What is the primary intent of the novella for ...,"[Taimoor, “Maybe but what difference does it m...",Search results: The primary intent of the nove...,The primary intent of the novella is to impart...,1.000000,0.275794,0.208333,0.568388,0.941987,0.773552,1.000000
2,Where Taimoor Ahmad study in the story?,[man trying to convince a scholar who knows al...,Search results: Taimoor Ahmad studies at Imper...,"Taimoor Ahmad studies at Imperial University, ...",1.000000,0.325000,0.200000,0.937459,0.905901,0.749834,1.000000
3,Who is Zaini in relation to Taimoor?,[man trying to convince a scholar who knows al...,"Based on the **scene summaries**, Taimoor meet...",Zaini is a very intelligent girl who used to p...,0.000000,0.000000,0.250000,0.737889,0.858218,0.701555,0.500000
4,Who is Gull's best friend?,[Gull encouraged and believed him that everyth...,Based on the **scene summaries** and **search ...,"Gull's best friend is Samia, who is also a cla...",0.000000,0.000000,0.333333,0.364598,0.702499,0.708392,0.800000
...,...,...,...,...,...,...,...,...,...,...,...
140,What were Taimoor's reasons for wanting to mee...,[June her friend Aliza was also in university ...,"Okay, let's analyze Taimoor's reasons for want...",Taimoor wanted to meet Hadia before the univer...,0.000000,0.666667,1.000000,0.498440,0.999225,0.743758,0.500000
141,What impact did Hadia's experiences at the uni...,[special and attracted him to Hadia. Although ...,"Okay, let's analyze the impact of Hadia's expe...","Hadia's experiences at the university, where s...",1.000000,0.709524,0.937925,0.744977,0.977257,0.798090,1.000000
142,What significant event related to Taimoor's fe...,[June her friend Aliza was also in university ...,Based on the **search results**:\n\nThe signif...,When Hadia announced she would come to univers...,1.000000,0.365476,0.302778,0.771557,1.000000,0.836227,1.000000
143,What role did Aliza play in Taimoor's situatio...,[and she (Aliza) was trying to misguide everyo...,"Okay, let's break down Aliza's role in Taimoor...",Aliza played a supportive role in Taimoor's si...,0.666667,0.608333,1.000000,0.378140,0.966303,0.738368,0.714286


In [79]:
scoreset_pd = scoreset_pd.iloc[scoreset_pd.user_input.drop_duplicates().index].copy()

In [80]:
scoreset_pd

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What is the publication date of the First Edit...,[Love you so…. Much Mr. T. (Taimoor’s nickname...,Search results: First Edition: May 2024\n,The First Edition is scheduled for May 2024.,1.000000,0.111111,0.111111,0.157921,0.733777,0.631684,NaN
1,What is the primary intent of the novella for ...,"[Taimoor, “Maybe but what difference does it m...",Search results: The primary intent of the nove...,The primary intent of the novella is to impart...,1.000000,0.275794,0.208333,0.568388,0.941987,0.773552,1.000000
2,Where Taimoor Ahmad study in the story?,[man trying to convince a scholar who knows al...,Search results: Taimoor Ahmad studies at Imper...,"Taimoor Ahmad studies at Imperial University, ...",1.000000,0.325000,0.200000,0.937459,0.905901,0.749834,1.000000
3,Who is Zaini in relation to Taimoor?,[man trying to convince a scholar who knows al...,"Based on the **scene summaries**, Taimoor meet...",Zaini is a very intelligent girl who used to p...,0.000000,0.000000,0.250000,0.737889,0.858218,0.701555,0.500000
4,Who is Gull's best friend?,[Gull encouraged and believed him that everyth...,Based on the **scene summaries** and **search ...,"Gull's best friend is Samia, who is also a cla...",0.000000,0.000000,0.333333,0.364598,0.702499,0.708392,0.800000
...,...,...,...,...,...,...,...,...,...,...,...
140,What were Taimoor's reasons for wanting to mee...,[June her friend Aliza was also in university ...,"Okay, let's analyze Taimoor's reasons for want...",Taimoor wanted to meet Hadia before the univer...,0.000000,0.666667,1.000000,0.498440,0.999225,0.743758,0.500000
141,What impact did Hadia's experiences at the uni...,[special and attracted him to Hadia. Although ...,"Okay, let's analyze the impact of Hadia's expe...","Hadia's experiences at the university, where s...",1.000000,0.709524,0.937925,0.744977,0.977257,0.798090,1.000000
142,What significant event related to Taimoor's fe...,[June her friend Aliza was also in university ...,Based on the **search results**:\n\nThe signif...,When Hadia announced she would come to univers...,1.000000,0.365476,0.302778,0.771557,1.000000,0.836227,1.000000
143,What role did Aliza play in Taimoor's situatio...,[and she (Aliza) was trying to misguide everyo...,"Okay, let's break down Aliza's role in Taimoor...",Aliza played a supportive role in Taimoor's si...,0.666667,0.608333,1.000000,0.378140,0.966303,0.738368,0.714286


In [81]:
scoreset_pd.iloc[:, 4:].mean()

context_recall                             0.716667
llm_context_precision_with_reference       0.552454
llm_context_precision_without_reference    0.738757
answer_correctness                         0.445329
answer_relevancy                           0.758409
semantic_similarity                        0.735324
faithfulness                               0.729444
dtype: float64

In [91]:
best_scoreset_pd = scoreset_pd[(scoreset_pd.iloc[:, 4:] > 0.5).mean(axis=1) == 1].reset_index(drop=True).copy()
filter_scoreset_pd = scoreset_pd[~scoreset_pd.response.str.contains('no mention')].reset_index(drop=True).copy()

In [92]:
best_scoreset_pd

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What qualities of Hadia are influencing Taimur...,[special and attracted him to Hadia. Although ...,Based on the **search results** and **scene su...,"Taimur is influenced by Hadia's soul, her beha...",1.000000,0.876667,1.000000,0.808238,0.988508,0.807419,1.000000
1,Wut is Taimur doin in the hostel and why is he...,"[restrictions. There was another thing, that n...",**Scene Summaries:** Taimur is in his hostel a...,Taimur was in the hostel and was missing Hadia...,1.000000,1.000000,0.505952,0.592764,0.866305,0.734692,0.571429
2,What emotions did Taimoor experience after the...,"[Saim called him and said, “If I tell you the ...",Based on the **search results**:\n\nAfter the ...,"After the call ended, Taimoor experienced sadn...",1.000000,0.732143,0.553571,0.611393,0.947832,0.778905,1.000000
3,Why Hadia tell Taimoor to not call and text he...,[On 10 July when he texted Saim and wanted to ...,**Search results:**\n\n* Hadia tells Taimoor...,Hadia told Taimoor to not call and text her ag...,1.000000,0.644104,0.771379,0.684196,0.806802,0.827694,0.846154
4,What advice does Gull give to Taimoor about pu...,"[Gull, “Listen Taimoor, the world is so cruel....",**Search results:**\nGull advises Taimoor to g...,Gull advises Taimoor that the world is cruel a...,1.000000,1.000000,0.833333,0.559799,0.924251,0.739197,1.000000
5,What significant event related to university o...,[as a lover or not. It was the second time he ...,**Search results:**\nNext week Hadia gave good...,The significant event related to university oc...,0.666667,0.709524,0.709524,0.537913,0.920141,0.767036,0.545455
6,What happened at the university when Hadia cel...,[another thing that he was a simple man in rea...,"**Search results:**\n\n* ""As the university ...","At the university, students were allowed in to...",1.000000,0.750000,0.966667,0.652116,0.977203,0.843759,0.750000
7,What events at the university led to Taimur's ...,"[After a quarter of a month, Taimur’s vacation...","Okay, let's analyze the events at the universi...",Taimur's anxiety attack at the university was ...,0.750000,0.736111,1.000000,0.557244,0.990796,0.780699,0.809524
8,What does Taimoor imply about the concept of o...,[encouraging oneself to become a better person...,"Okay, let's analyze what Taimoor implies about...",Taimoor implies that the concept of oneness in...,0.666667,0.942857,0.948413,0.668930,0.987706,0.818578,0.529412
9,What was Taimur's situation regarding his heal...,[It was almost 07:00 in the evening on 27th Ju...,Based on the **scene summaries** and **relevan...,Taimur faced a serious anxiety attack due to h...,0.666667,0.916667,0.583333,0.603262,0.906005,0.698761,1.000000


In [93]:
filter_scoreset_pd

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What is the publication date of the First Edit...,[Love you so…. Much Mr. T. (Taimoor’s nickname...,Search results: First Edition: May 2024\n,The First Edition is scheduled for May 2024.,1.000000,0.111111,0.111111,0.157921,0.733777,0.631684,NaN
1,What is the primary intent of the novella for ...,"[Taimoor, “Maybe but what difference does it m...",Search results: The primary intent of the nove...,The primary intent of the novella is to impart...,1.000000,0.275794,0.208333,0.568388,0.941987,0.773552,1.000000
2,Where Taimoor Ahmad study in the story?,[man trying to convince a scholar who knows al...,Search results: Taimoor Ahmad studies at Imper...,"Taimoor Ahmad studies at Imperial University, ...",1.000000,0.325000,0.200000,0.937459,0.905901,0.749834,1.000000
3,Who is Zaini in relation to Taimoor?,[man trying to convince a scholar who knows al...,"Based on the **scene summaries**, Taimoor meet...",Zaini is a very intelligent girl who used to p...,0.000000,0.000000,0.250000,0.737889,0.858218,0.701555,0.500000
4,Who is Gull's best friend?,[Gull encouraged and believed him that everyth...,Based on the **scene summaries** and **search ...,"Gull's best friend is Samia, who is also a cla...",0.000000,0.000000,0.333333,0.364598,0.702499,0.708392,0.800000
...,...,...,...,...,...,...,...,...,...,...,...
134,What were Taimoor's reasons for wanting to mee...,[June her friend Aliza was also in university ...,"Okay, let's analyze Taimoor's reasons for want...",Taimoor wanted to meet Hadia before the univer...,0.000000,0.666667,1.000000,0.498440,0.999225,0.743758,0.500000
135,What impact did Hadia's experiences at the uni...,[special and attracted him to Hadia. Although ...,"Okay, let's analyze the impact of Hadia's expe...","Hadia's experiences at the university, where s...",1.000000,0.709524,0.937925,0.744977,0.977257,0.798090,1.000000
136,What significant event related to Taimoor's fe...,[June her friend Aliza was also in university ...,Based on the **search results**:\n\nThe signif...,When Hadia announced she would come to univers...,1.000000,0.365476,0.302778,0.771557,1.000000,0.836227,1.000000
137,What role did Aliza play in Taimoor's situatio...,[and she (Aliza) was trying to misguide everyo...,"Okay, let's break down Aliza's role in Taimoor...",Aliza played a supportive role in Taimoor's si...,0.666667,0.608333,1.000000,0.378140,0.966303,0.738368,0.714286


In [94]:
print('Len best dataset:', len(best_scoreset_pd))
print('Len good dataset:', len(filter_scoreset_pd))

Len best dataset: 14
Len good dataset: 139


In [95]:
best_scoreset_pd.iloc[:, 4:].mean()

context_recall                             0.910714
llm_context_precision_with_reference       0.850185
llm_context_precision_without_reference    0.833240
answer_correctness                         0.655408
answer_relevancy                           0.919518
semantic_similarity                        0.777893
faithfulness                               0.831755
dtype: float64

In [96]:
filter_scoreset_pd.iloc[:, 4:].mean()

context_recall                             0.736811
llm_context_precision_with_reference       0.565137
llm_context_precision_without_reference    0.768138
answer_correctness                         0.455837
answer_relevancy                           0.786314
semantic_similarity                        0.736478
faithfulness                               0.727190
dtype: float64

In [100]:
def print_score_pd(scoreset_pd, idx):
    print('user_input:', scoreset_pd.iloc[idx]['user_input'], '\n')
    print('retrieved_contexts:', scoreset_pd.iloc[idx]['retrieved_contexts'], '\n')
    print('response:', scoreset_pd.iloc[idx]['response'], '\n')
    print('reference:', scoreset_pd.iloc[idx]['reference'], '\n')
    
    print('context_recall =', scoreset_pd.iloc[idx]['context_recall'])
    print('llm_context_precision_with_reference =', scoreset_pd.iloc[idx]['llm_context_precision_with_reference'])
    print('llm_context_precision_without_reference =', scoreset_pd.iloc[idx]['llm_context_precision_without_reference'])
    print('answer_correctness =', scoreset_pd.iloc[idx]['answer_correctness'])
    print('answer_relevancy =', scoreset_pd.iloc[idx]['answer_relevancy'])
    print('semantic_similarity =', scoreset_pd.iloc[idx]['semantic_similarity'])
    print('faithfulness =', scoreset_pd.iloc[idx]['faithfulness'])

In [101]:
print_score_pd(filter_scoreset_pd, 70)

user_input: What challenges did Taimur face regarding his feelings for Hadia while attending the university? 

retrieved_contexts: ['After a quarter of a month, Taimur’s vacation ended, and he\r\nstarted his internship, but Hadia was still on leave. He was\r\nwaiting for her whenever he visited the university. In these days,\r\nTaimur again insisted on her but the matter remained unresolved.\r\nAgain, Taimur blocked her and another person, Sameer who was\r\na mutual friend, explained the whole situation to Taimur that she\r\nwept in front of him, but he was not clear whether she liked him', 'After a few days of internship, Taimur attracted more towards Hadia\r\nand again tried to convince her but at this time he was very confused\r\nand upset.\r\nHe was thinking “If today she will deny, it means she will never give\r\nme a chance to spend life with her”.\r\nThat was the main thing that was haunting his mind at that time.\r\n\r\n                                                          

In [102]:
print_score_pd(filter_scoreset_pd, 5)

user_input: How did Hadia and Taimur first meet? 

retrieved_contexts: ['as a lover or not. It was the second time he unblocked her.\r\n\r\nNow, it seemed that Hadia was also absorbed in him, but it was\r\nnot sured. Next week Hadia gave good news to Taimur, “I will\r\ncome to university by tomorrow”.\r\n\r\nIt was a piece of breaking news for Taimur. He was very happy\r\nafter seeing that text.\r\n\r\nAlthough, they already met, but now there was another thing\r\nwhich was a heart-full connection from Taimur. But there was', 'due to Taimur’s shyness, hesitation or perhaps he preferred\r\nformal conversation. But Hadia was surprised at the question and\r\ngave him a cute smile. They both were very nervous because of\r\ntheir first meeting. But she nodded. Now, Taimur had no words\r\nto speak and almost finished his meeting. But Sameer came near\r\nthem and started talking to them.\r\n\r\nNow, Taimur presented another treat to both Hadia and Sameer\r\nwhich was a cold drink with chocola